In [28]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

LR=0.001
EPOCHS=100

In [29]:
device

device(type='cpu')

In [30]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train = ImageFolder(root='./train', transform=transform)
test = ImageFolder(root='./test', transform=transform)
val = ImageFolder(root='./val', transform=transform)

train_dl = DataLoader(train, batch_size=512, shuffle=True, drop_last=True)
test_dl = DataLoader(test, batch_size=128, shuffle=True, drop_last=True)
val_dl = DataLoader(val, batch_size=128, shuffle=True, drop_last=True)

In [31]:
class FCN(nn.Module):
  def __init__(self, in_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(in_features, 64),
        nn.Linear(64, 64),
        nn.Linear(64, 64),
        nn.Linear(64, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.model(x)

In [32]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(num_features=32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
        nn.BatchNorm2d(num_features=64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        nn.BatchNorm2d(num_features=128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),

        nn.AdaptiveAvgPool2d(output_size=(2,2)),
        nn.Flatten(),
        nn.Linear(128*2*2, 64),
        nn.Linear(64, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.model(x)

In [33]:
class ResNetModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.resnet = models.resnet50(pretrained=True)
    for param in self.resnet.parameters():
      param.requires_grad = False

    self.resnet.fc = nn.Sequential(
        nn.Linear(self.resnet.fc.in_features, 1),
        nn.Sigmoid()
    )
  
  def forward(self, x):
    return self.resnet(x)

In [34]:
class VGG19Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.vgg19 = models.vgg19(pretrained=True)
    for param in self.vgg19.parameters():
      param.requires_grad = False
    self.vgg19.classifier[-1] = nn.Sequential(
        nn.Linear(self.vgg19.classifier[-1].in_features, 1),
        nn.Sigmoid()
    )
  
  def forward(self, x):
    return self.vgg19(x)

In [37]:
import tqdm

def train_function(model, train_dataloader, val_dataloader, loss_fn, device, fcn=False):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    for epoch in tqdm.tqdm(range(EPOCHS)):
        train_loss = 0.0
        val_loss = 0.0
        train_total = 0
        val_total = 0
        train_correct = 0
        val_correct = 0

        model.train()
        for images, labels in train_dataloader:
          images, labels = images.to(device), labels.to(device)
          labels = labels.to(torch.float32)
          if fcn:
            images = images.view(images.shape[0], -1)
          optimizer.zero_grad()
          outputs = model(images)
          loss = loss_fn(outputs.squeeze(1), labels)
          loss.backward()
          optimizer.step()

          train_loss += loss.item() * images.size(0)
          train_total += labels.size(0)
          _, predicted = torch.max(outputs.data, 1)
          train_correct += (predicted == labels).sum().item()

        model.eval()
        with torch.no_grad():
          for images, labels in val_dataloader:
            labels = labels.to(torch.float32)
            images, labels = images.to(device), labels.to(device)
            if fcn:
              images = images.view(images.shape[0], -1)
            outputs = model(images)
            loss = loss_fn(outputs.squeeze(1), labels)

            val_loss += loss.item() * images.size(0)
            val_total += labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

        train_losses.append(train_loss / len(train_dataloader.dataset))
        val_losses.append(val_loss / len(val_dataloader.dataset))
        train_accs.append(train_correct / train_total)
        val_accs.append(val_correct / val_total)

        if epoch % 10 == 0:
          print('Epoch [{}/{}], Train Loss: {:.4f}, Val Loss: {:.4f}, Train Acc: {:.4f}, Val Acc: {:.4f}'.format(epoch+1, EPOCHS, train_losses[-1], val_losses[-1], train_accs[-1], val_accs[-1]))

    return train_losses, val_losses, train_accs, val_accs

In [38]:
fcn_model = FCN(in_features=3*64*64).to(device)
cnn_model = CNN().to(device)
resnet_model = ResNetModel().to(device)
vgg19_model = VGG19Model().to(device)

loss_fn = nn.BCELoss()

c:\Users\tolst\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tolst\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\tolst\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGEN

In [39]:
fcn_train_losses, fcn_val_losses, fcn_train_accs, fcn_val_accs = train_function(model = fcn_model, 
                                                                                train_dataloader=train_dl,
                                                                                val_dataloader=val_dl,
                                                                                loss_fn=loss_fn,
                                                                                device=device,
                                                                                fcn=True)

  1%|          | 1/100 [00:37<1:01:06, 37.04s/it]

Epoch [1/100], Train Loss: 0.7291, Val Loss: 0.6550, Train Acc: 0.5003, Val Acc: 0.4996


 11%|█         | 11/100 [06:15<49:41, 33.50s/it] 

Epoch [11/100], Train Loss: 0.6315, Val Loss: 0.6663, Train Acc: 0.4998, Val Acc: 0.4988


 21%|██        | 21/100 [11:51<44:31, 33.82s/it]

...

 81%|████████  | 81/100 [44:52<09:49, 31.05s/it]

Epoch [81/100], Train Loss: 0.5676, Val Loss: 0.7021, Train Acc: 0.4999, Val Acc: 0.4996


 91%|█████████ | 91/100 [50:04<04:40, 31.11s/it]

Epoch [91/100], Train Loss: 0.5638, Val Loss: 0.7270, Train Acc: 0.5001, Val Acc: 0.4988


100%|██████████| 100/100 [54:43<00:00, 32.83s/it]


In [40]:
cnn_train_losses, cnn_val_losses, cnn_train_accs, cnn_val_accs = train_function(model = cnn_model, 
                                                                                train_dataloader=train_dl,
                                                                                val_dataloader=val_dl,
                                                                                loss_fn=loss_fn,
                                                                                device=device)

  1%|          | 1/100 [01:28<2:26:30, 88.80s/it]

Epoch [1/100], Train Loss: 0.6036, Val Loss: 0.5649, Train Acc: 0.4999, Val Acc: 0.4963


 11%|█         | 11/100 [16:12<2:11:07, 88.40s/it]

Epoch [11/100], Train Loss: 0.2541, Val Loss: 0.3688, Train Acc: 0.4999, Val Acc: 0.4971


 21%|██        | 21/100 [32:00<2:06:28, 96.05s/it]

...

 81%|████████  | 81/100 [2:09:40<29:57, 94.62s/it]

Epoch [81/100], Train Loss: 0.0003, Val Loss: 0.7365, Train Acc: 0.5002, Val Acc: 0.4971


 91%|█████████ | 91/100 [2:26:29<14:55, 99.53s/it] 

Epoch [91/100], Train Loss: 0.0002, Val Loss: 0.7548, Train Acc: 0.4998, Val Acc: 0.4984


100%|██████████| 100/100 [2:41:21<00:00, 96.81s/it]


In [41]:
resnet_train_losses, resnet_val_losses, resnet_train_accs, resnet_val_accs = train_function(model = resnet_model, 
                                                                                            train_dataloader=train_dl,
                                                                                            val_dataloader=val_dl ,
                                                                                            loss_fn=loss_fn,
                                                                                            device=device)

  1%|          | 1/100 [02:28<4:05:10, 148.59s/it]

Epoch [1/100], Train Loss: 0.5337, Val Loss: 0.4504, Train Acc: 0.5000, Val Acc: 0.4967


 11%|█         | 11/100 [27:19<3:42:54, 150.28s/it]

Epoch [11/100], Train Loss: 0.3973, Val Loss: 0.4060, Train Acc: 0.4998, Val Acc: 0.4963


 21%|██        | 21/100 [52:26<3:17:49, 150.25s/it]

...

 81%|████████  | 81/100 [3:23:16<42:53, 135.44s/it]  

Epoch [81/100], Train Loss: 0.3485, Val Loss: 0.4093, Train Acc: 0.4999, Val Acc: 0.4988


 91%|█████████ | 91/100 [3:45:45<20:13, 134.81s/it]

Epoch [91/100], Train Loss: 0.3454, Val Loss: 0.4103, Train Acc: 0.4999, Val Acc: 0.4979


100%|██████████| 100/100 [4:05:58<00:00, 147.59s/it]


In [42]:
vgg19_train_losses, vgg19_val_losses, vgg19_train_accs, vgg19_val_accs = train_function(model = vgg19_model, 
                                                                                        train_dataloader=train_dl,
                                                                                        val_dataloader=val_dl,
                                                                                        loss_fn=loss_fn,
                                                                                        device=device)

  1%|          | 1/100 [04:46<7:52:17, 286.23s/it]

Epoch [1/100], Train Loss: 0.3792, Val Loss: 0.2772, Train Acc: 0.4999, Val Acc: 0.4959


 11%|█         | 11/100 [52:15<7:01:41, 284.28s/it]

Epoch [11/100], Train Loss: 0.2631, Val Loss: 0.2438, Train Acc: 0.4998, Val Acc: 0.4967


 21%|██        | 21/100 [1:38:31<6:00:14, 273.60s/it]

...

 41%|████      | 41/100 [3:17:08<5:35:36, 341.30s/it]

Epoch [41/100], Train Loss: 0.2605, Val Loss: 0.2445, Train Acc: 0.5002, Val Acc: 0.4984


 45%|████▌     | 45/100 [3:35:19<4:24:53, 288.98s/it]

: 

: 

In [2]:
import json
import plotly.graph_objects as go
import numpy as np

with open('results.json', 'r') as f:
    data = json.load(f)

In [13]:
fig = go.Figure()
fig.add_traces(go.Scatter(x=np.arange(0, 10), y = data['Linear']['train_losses'], name="Линейная регрессия"))
fig.add_traces(go.Scatter(x=np.arange(0, 10), y = data['CNN']['train_losses'], name="Свёрточная сеть"))
fig.add_traces(go.Scatter(x=np.arange(0, 10), y = data["ResNet50"]['train_losses'], name="ResNet"))
fig.add_traces(go.Scatter(x=np.arange(0, 10), y = data['VGG19']['train_losses'], name="VGG19"))
fig.update_layout(xaxis_title='Эпохи', yaxis_title='Ошибка')
fig.show()